In [9]:
import pandas as pd
import numpy as np

from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

In [10]:
train = pd.read_csv("C:\\Users\\solba\\dacon-project\\data\\raw\\train.csv")
test = pd.read_csv("C:\\Users\\solba\\dacon-project\\data\\raw\\test.csv")

In [11]:
columns_to_drop = ['generation', 'contest_award', 'completed_semester', 
                   'incumbents_lecture_scale_reason', 'interested_company', 'contest_participation', 'idea_contest']
train = train.drop(columns=columns_to_drop)
print(f"삭제 후 train shape: {train.shape}")

삭제 후 train shape: (748, 39)


"""결측치 null로 채우기"""

In [12]:
# 결측치 확인
print("결측치 개수:")
print(train.isnull().sum())
print(f"\n전체 결측치: {train.isnull().sum().sum()}")
print(f"\n결측치 비율:\n{train.isnull().sum() / len(train) * 100}")

결측치 개수:
ID                            0
school1                       0
major type                   22
major1_1                     20
major1_2                    439
major_data                    0
job                           0
class1                        0
class2                      579
class3                      734
class4                      747
re_registration               0
nationality                   1
inflow_route                  0
whyBDA                        0
what_to_gain                  0
hope_for_group                0
previous_class_3            602
previous_class_4            602
previous_class_5            602
previous_class_6            602
previous_class_7            602
previous_class_8            602
major_field                  23
desired_career_path           0
project_type                  0
time_input                    0
desired_job                   0
certificate_acquisition       0
desired_certificate           0
desired_job_except_data       0


In [13]:
# 모든 컬럼을 category형으로 변환 (completed 제외)
# float 값을 string으로 먼저 변환 후 category로 변환
for column in train.columns:
    if column != 'completed':
        # float나 int 값을 모두 string으로 변환 후 category로
        train[column] = train[column].astype(str).astype('category')
    else:
        train[column] = train[column].astype('int')

print("컬럼 타입 변환 완료!")
print(f"\n데이터 타입 확인:")
print(train.dtypes)

컬럼 타입 변환 완료!

데이터 타입 확인:
ID                          category
school1                     category
major type                  category
major1_1                    category
major1_2                    category
major_data                  category
job                         category
class1                      category
class2                      category
class3                      category
class4                      category
re_registration             category
nationality                 category
inflow_route                category
whyBDA                      category
what_to_gain                category
hope_for_group              category
previous_class_3            category
previous_class_4            category
previous_class_5            category
previous_class_6            category
previous_class_7            category
previous_class_8            category
major_field                 category
desired_career_path         category
project_type                category
time_input   

In [14]:
for column in train.columns:
    if train[column].isnull().sum() > 0:
        train[column].fillna('Unknown', inplace=True)
print("결측치 처리 완료!")
print(f"남은 결측치: {train.isnull().sum().sum()}")

결측치 처리 완료!
남은 결측치: 0


In [15]:
# completed의 분포 확인
print("오버샘플링 전:")
print(train['completed'].value_counts())
print(f"비율: {train['completed'].value_counts(normalize=True)}")

# completed가 1인 행만 추출
completed_1 = train[train['completed'] == 1]

# completed가 1인 행을 복제해서 train에 추가
train_oversampled = pd.concat([train, completed_1], ignore_index=True)

# 데이터 셔플
train_oversampled = train_oversampled.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\n오버샘플링 후:")
print(train_oversampled['completed'].value_counts())
print(f"비율: {train_oversampled['completed'].value_counts(normalize=True)}")
print(f"\n데이터 shape: {train_oversampled.shape}")

# train을 오버샘플링된 데이터로 교체
train = train_oversampled

오버샘플링 전:
completed
0    525
1    223
Name: count, dtype: int64
비율: completed
0    0.701872
1    0.298128
Name: proportion, dtype: float64

오버샘플링 후:
completed
0    525
1    446
Name: count, dtype: int64
비율: completed
0    0.54068
1    0.45932
Name: proportion, dtype: float64

데이터 shape: (971, 39)


In [16]:
# ID 열 제외하고 X, y 분리
X = train.drop(['ID', 'completed'], axis=1)
y = train['completed']

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\n특성 목록:\n{X.columns.tolist()}")

X shape: (971, 37)
y shape: (971,)

특성 목록:
['school1', 'major type', 'major1_1', 'major1_2', 'major_data', 'job', 'class1', 'class2', 'class3', 'class4', 're_registration', 'nationality', 'inflow_route', 'whyBDA', 'what_to_gain', 'hope_for_group', 'previous_class_3', 'previous_class_4', 'previous_class_5', 'previous_class_6', 'previous_class_7', 'previous_class_8', 'major_field', 'desired_career_path', 'project_type', 'time_input', 'desired_job', 'certificate_acquisition', 'desired_certificate', 'desired_job_except_data', 'incumbents_level', 'incumbents_lecture', 'incumbents_company_level', 'incumbents_lecture_type', 'incumbents_lecture_scale', 'expected_domain', 'onedayclass_topic']


In [19]:
cat_features = list(range(len(X.columns)))

In [21]:
train.head(3)

,ID,school1,major type,major1_1,major1_2,major_data,job,class1,class2,class3,...,desired_certificate,desired_job_except_data,incumbents_level,incumbents_lecture,incumbents_company_level,incumbents_lecture_type,incumbents_lecture_scale,expected_domain,onedayclass_topic,completed
0,TRAIN_168,27,"복수 전공 ( 다중전공, 이중전공 포함 )",경영학,자연과학,True,대학생,8,12.0,nan,...,"ADsP, 빅데이터 분석 기사, 태블로 관련 자격증","A. 금융 / 보험 직무, B. 기획 / 전략 / 경영 직무",주니어 (0~3년차),직무 강의 (예시: 실무 진행 방식 및 직무 준비생을 위한 팁),국내 빅테크 IT 계열 (네카라쿠배당토),"온, 오프라인 동시",3~50명 내외의 강의 리스너와 1명의 현직자,"J. 정보통신업, K. 금융 및 보험업","Python 응용, 데이터 시각화 (Matplotlib, Seaborn 등), SQ...",0
1,TRAIN_302,27,"복수 전공 ( 다중전공, 이중전공 포함 )",인문학,자연과학,False,대학생,12,nan,nan,...,"ADsP, SQLD, 빅데이터 분석 기사",A. 금융 / 보험 직무,주니어 (0~3년차),커리어 패스 과정 (예시: 비전공자/전공자의 취업 준비 및 이직 과정),은행 및 보험 쪽 IT,오프라인,10명 내외의 강의 리스너와 1명의 현직자,K. 금융 및 보험업,"데이터 시각화 (Matplotlib, Seaborn 등), SQL 응용",1
2,TRAIN_076,17,"복수 전공 ( 다중전공, 이중전공 포함 )",IT(컴퓨터 공학 포함),IT(컴퓨터 공학 포함),True,대학생,1,nan,nan,...,"ADsP, SQLD, 빅데이터 분석 기사, 정보처리기사, 구글 애널리스트",D. 소프트웨어 개발자,주니어 (0~3년차),직무 강의 (예시: 실무 진행 방식 및 직무 준비생을 위한 팁),해외 기업 (빅테크),"온, 오프라인 동시",100명 이상의 리스너와 10명 이상의 현직자,"K. 금융 및 보험업, U. 국제 및 외국기관","데이터 시각화 (Matplotlib, Seaborn 등), 머신러닝 / 딥러닝 응용",0


In [20]:
cat_features

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36]

In [ ]:
model = CatBoostRegressor(
    iterations=1000, 
    depth=6,    
    learning_rate=0.01, 
    loss_function='RMSE', 
    cat_features=cat_features,
    l2_leaf_reg=5,
    random_seed=42,
    verbose=100, 
    
    task_type="GPU",
    devices='0'
)

print("모델 학습 시작... (GPU 모드)")

model.fit(X, y)

print("\n모델 학습 완료!")

모델 학습 시작... (GPU 모드)
0:	learn: 0.4916115	total: 48.6ms	remaining: 48.6s
200:	learn: 0.0814546	total: 5.56s	remaining: 22.1s
400:	learn: 0.0674621	total: 10.4s	remaining: 15.5s
600:	learn: 0.0638678	total: 15.2s	remaining: 10.1s
800:	learn: 0.0596986	total: 20.1s	remaining: 4.99s
999:	learn: 0.0579599	total: 25s	remaining: 0us

모델 학습 완료!


In [18]:
# 학습 결과 확인
y_pred = model.predict(X)

# RMSE, R2 Score
rmse = np.sqrt(mean_squared_error(y, y_pred))
r2 = r2_score(y, y_pred)

# Accuracy: 0.5 기준으로 0/1 분류
y_pred_binary = (y_pred >= 0.5).astype(int)
accuracy = accuracy_score(y, y_pred_binary)

print(f"Train RMSE: {rmse:.4f}")
print(f"Train R2 Score: {r2:.4f}")
print(f"Train Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

# 특성 중요도 확인
feature_importance = model.get_feature_importance()
feature_names = X.columns

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print(f"\n특성 중요도 Top 10:")
print(importance_df.head(10))

Train RMSE: 0.0613
Train R2 Score: 0.9849
Train Accuracy: 0.9979 (99.79%)

특성 중요도 Top 10:
                    feature  importance
0                   school1   31.073013
26              desired_job   23.628476
35          expected_domain   23.588125
28      desired_certificate   11.756745
36        onedayclass_topic    2.621547
29  desired_job_except_data    1.939883
33  incumbents_lecture_type    1.644280
27  certificate_acquisition    1.087099
6                    class1    0.364349
13                   whyBDA    0.311660
